## Sample structured dataset

We will add two tables in a Glue database names "retail." Firstly, two sample CSV files will be loaded to a S3 bucket then new external tables will be created using Amazon Athena.

Two tables are added in this notebook:
* orders
* returns


#### Prerequites
* Create an IAM service role for Amazon LakeFormation integration

In [ ]:
%pip install -r requirements.txt --quiet --upgrade

In [ ]:
# Variables
import boto3

region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
s3_bucket_name = f"{account_id}-{region}-s3-retail-db-for-agent-example"
db_name = "retail"
s3_output = f"s3://{s3_bucket_name}/athena-query-output/"

account_id	 boto3	 db_name	 region	 s3_bucket_name	 s3_output	 


#### Create a S3 bucket

In [ ]:
# Step1: Createa a S3 table bucket
import boto3
s3 = boto3.client('s3')
try:
    response = s3.create_bucket(Bucket=s3_bucket_name)
    print("S3 bucket created")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("S3 bucket bucket already exists")
    # Clean up
    #s3tables.delete_table_bucket(tableBucketARN=s3table_bucket_arn)

S3 bucket created


#### Upload data

In [3]:
# A helper functino to upload CSV files from Excel files.
import boto3
import pandas as pd
from io import BytesIO

def load_excel_to_df(source_bucket, source_key):
    """
    Convert Excel file from S3 to CSV and save back to S3.
    
    Args:
        source_bucket (str): Source S3 bucket name
        source_key (str): Source file key (path to xlsx file)
        target_bucket (str): Target S3 bucket name
        target_prefix (str): Target prefix (folder) for CSV files
    """
    # Initialize S3 client
    s3_client = boto3.client('s3')
    # Read the Excel file from S3
    response = s3_client.get_object(Bucket=source_bucket, Key=source_key)

    excel_data = response['Body'].read()

    # Load Excel file into pandas
    excel_file = pd.ExcelFile(BytesIO(excel_data))

    df = pd.read_excel(excel_file, sheet_name=excel_file.sheet_names[0])
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    print(df.head(5))

    return df


print("Function load_excel_to_df has been created")

Function load_excel_to_df has been created


#### Create a Glue Database

In [4]:
try:
    # Create Glue client
    glue_client = boto3.client('glue')
    
    # Prepare database input parameters
    database_input = {
        'Name': db_name
    }
    
    # Create the database
    response = glue_client.create_database(
        DatabaseInput=database_input
    )
    
    print(f"Successfully created database: {db_name}")
    
except glue_client.exceptions.AlreadyExistsException:
        print(f"Database {db_name} already exists")


Database retail already exists


In [5]:
# Load data in to DataFrames
order_table_s3_key = "artifacts/aws-blog-joining-across-quicksight/orders.xlsx"
returns_table_s3_key = "artifacts/aws-blog-joining-across-quicksight/returns.xlsx"
source_bucket_name = "aws-bigdata-blog"


df_orders = load_excel_to_df(source_bucket_name, order_table_s3_key)
df_returns = load_excel_to_df(source_bucket_name, returns_table_s3_key)

df_orders.to_csv(f"s3://{s3_bucket_name}/data/orders/orders.csv", index=False)
df_returns.to_csv(f"s3://{s3_bucket_name}/data/returns/returns.csv", index=False)

   row_id         order_id order_date  ship_date     ship_mode customer_id  \
0   32298   CA-2012-124891 2012-07-31 2012-07-31      Same Day    RH-19495   
1   26341    IN-2013-77878 2013-02-05 2013-02-07  Second Class    JR-16210   
2   25330    IN-2013-71249 2013-10-17 2013-10-18   First Class    CR-12730   
3   13524  ES-2013-1579342 2013-01-28 2013-01-30   First Class    KM-16375   
4   47221     SG-2013-4320 2013-11-05 2013-11-06      Same Day     RH-9495   

      customer_name      segment           city            state  ...  \
0       Rick Hansen     Consumer  New York City         New York  ...   
1     Justin Ritter    Corporate     Wollongong  New South Wales  ...   
2      Craig Reiter     Consumer       Brisbane       Queensland  ...   
3  Katherine Murray  Home Office         Berlin           Berlin  ...   
4       Rick Hansen     Consumer          Dakar            Dakar  ...   

         product_id    category sub-category  \
0   TEC-AC-10003033  Technology  Accessories

/Users/dggallit/Documents/agentic-workflows-using-open-source-reasoning-model-on-amazon-web-services/.venv/lib/python3.12/site-packages/fsspec/registry.py:286: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


#### Create external tables

In [6]:
# SQL

orders_sql = f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS orders (
        row_id BIGINT,
        order_id STRING,
        order_date TIMESTAMP,
        ship_date TIMESTAMP,
        ship_mode STRING,
        customer_id STRING,
        customer_name STRING,
        segment STRING,
        city STRING,
        state STRING,
        country STRING,
        postal_code DOUBLE,
        market STRING,
        region STRING,
        product_id STRING,
        category STRING,
        `sub-category` STRING,
        product_name STRING,
        sales DOUBLE,
        quantity BIGINT,
        discount DOUBLE,
        profit DOUBLE,
        shipping_cost DOUBLE,
        order_priority STRING
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    LINES TERMINATED BY '\n'
    LOCATION 's3://{account_id}-{region}-s3-retail-db-for-agent-example/data/orders'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

returns_sql = f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS returns (
        returned STRING,
        order_id STRING,
        market STRING
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    LINES TERMINATED BY '\n'
    LOCATION 's3://{account_id}-{region}-s3-retail-db-for-agent-example/data/returns'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

In [7]:
# A helper function
import boto3
import time
import pandas as pd
from IPython.display import display

def execute_athena_query(query, database=db_name, s3_output=s3_output):
    # Create an Athena client
    athena_client = boto3.client('athena')
    
    # Start the query execution
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': s3_output
        }
    )
    
    # Get the query execution ID
    query_execution_id = response['QueryExecutionId']
    
    # Wait for the query to complete
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        state = response['QueryExecution']['Status']['State']
        
        if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break
            
        time.sleep(1)  # Wait for 1 second before checking again
    
    # If query succeeded, get the results
    if state == 'SUCCEEDED':
        # Initialize paginator
        results = []
        paginator = athena_client.get_paginator('get_query_results')
        for page in paginator.paginate(QueryExecutionId=query_execution_id):
            for row in page['ResultSet']['Rows']:
                results.append([field.get('VarCharValue', '') for field in row['Data']])
                 
        # Get results
        if results:
            df = pd.DataFrame(results[1:], columns = results[0])
            display(df)
        else:
            print("Successfully executed.")
    else:
        error_message = response['QueryExecution']['Status'].get('StateChangeReason', 'No error message available')
        raise Exception(f"Query failed: {error_message}")
print("Function created")

Function created


In [8]:
execute_athena_query(orders_sql)
execute_athena_query(returns_sql)

Successfully executed.
Successfully executed.


#### Validate data

In [9]:
execute_athena_query(
    "SELECT * FROM orders LIMIT 5"
)
execute_athena_query(
    "SELECT * FROM returns LIMIT 5"
)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,32298,CA-2012-124891,,,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.65,7,0.0,762.1844999999998,933.57,Critical
1,26341,IN-2013-77878,,,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"""Novimex Executive Leather Armchair",,3709,9.0,0.1,-288.765,923.63
2,25330,IN-2013-71249,,,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"""Nokia Smart Phone",,5175,9.0,0.1,919.9709999999997,915.49
3,13524,ES-2013-1579342,,,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"""Motorola Smart Phone",,2892,5.0,0.1,-96.54000000000005,910.16
4,47221,SG-2013-4320,,,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"""Sharp Wireless Fax",,2832,8.0,0.0,311.52,903.04


,returned,order_id,market
0,Yes,MX-2013-168137,LATAM
1,Yes,US-2011-165316,LATAM
2,Yes,ES-2013-1525878,EU
3,Yes,CA-2013-118311,United States
4,Yes,ES-2011-1276768,EU
